# Datos Climáticos Estado/Municipio NASA API

El proyecto **"Prediction Of Worldwide Energy Resources (POWER)"** de la **NASA** se inició con el propósito de mejorar los conjuntos de datos relacionados con energía renovable y crear nuevos conjuntos de datos a partir de sistemas de satélites más recientes. Este proyecto tiene como objetivo servir a tres comunidades de usuarios: **(1)** la comunidad de Energía Renovable, **(2)** la de Edificios Sostenibles y **(3)** la de Agroclimatología.

El proyecto **POWER** proporciona conjuntos de datos solares y meteorológicos derivados de la investigación de la NASA para satisfacer las necesidades relacionadas con la energía renovable, la eficiencia energética de los edificios y la agricultura. Estos datos son valiosos debido a su frecuente actualización con un retraso de tan solo un día, lo que los hace disponibles en *"tiempo casi real"*.

Para más información visite el siguiente enlace: https://power.larc.nasa.gov/#resources

Para facilitar el acceso a estos datos, el proyecto POWER ofrece varias opciones:

- Data Access Viewer: Se trata de una aplicación de mapeo web receptiva que proporciona herramientas para subconjuntos de datos, gráficos y visualización en una interfaz fácil de usar.

- API Services Catalog: Este catálogo permite el acceso a los datos de POWER a través de scripts personalizados y aplicaciones escalables, lo que brinda flexibilidad a los usuarios.

- ArcGIS Image Services: Ofrece servicios de imágenes ArcGIS habilitados para el tiempo, lo que permite su uso en sistemas de información geográfica (GIS) para mapeo, visualización y análisis espacial.

- AWS Open Data Registry: El proyecto POWER pone a disposición su tienda de datos en formato NetCDF y Zarr en la nube a través de Amazon Web Services (AWS), lo que facilita el acceso a estos datos desde la nube.

Además de los conjuntos de datos, el proyecto POWER también ofrece recursos adicionales, como notas de lanzamiento, publicaciones, preguntas frecuentes y un foro de usuarios de Earthdata. Estos recursos adicionales ayudan a los usuarios a comprender y utilizar eficazmente los datos proporcionados por el proyecto.

<p align="center">
  <img src="power_logo_event.png" alt="Descripción de la imagen" width="40%" height="40%">
</p>

<p align="center">
  <img src="https://github.com/Axel-Castro/Resource/blob/main/Img/power_logo_event.png" alt="Descripción de la imagen" width="40%" height="40%">
</p>

<p align="center">
  <em>Figura 1: Prediction Of Worldwide Energy Resources (POWER).</em>
</p>

### Importación de librerías

In [48]:
import pandas as pd
import requests
import datetime
import csv
import os

### Creación de directorios

In [43]:
subdir = "./data/"

input_coord_estado = 'data/Data_In/Coordenadas_Centroide_Estado.csv' # Dir de archivo de entrada coordenadas estado
input_coord_municipio = 'data/Data_In/Coordenadas_Centroide_Municipio.csv' # Dir de archivo de entrada coordenadas municipio

output_estado_raw = "data/Estados/Estados_Raw/" # Dir de carpeta de salida estado raw
output_estado_mod = 'data/Estados/Estados_Mod/' # Dir de carpeta de salida estado mod

output_municipio_raw = "data/Municipios/Municipios_Raw/" # Dir de carpeta de salida municipio raw
output_municipio_mod = 'data/Municipios/Municipios_Mod' # Dir de carpeta de salida municipio mod

output_estado = 'data/Estados'
output_municipio = 'data/Municipios'

# Crea la carpeta de salida si no existe
if not os.path.exists(output_estado_raw):
    os.makedirs(output_estado_raw)

if not os.path.exists(output_estado_mod):
    os.makedirs(output_estado_mod)

if not os.path.exists(output_municipio_raw):
    os.makedirs(output_municipio_raw)

if not os.path.exists(output_municipio_mod):
    os.makedirs(output_municipio_mod)


### Creación de Registro

In [47]:
with open(subdir + "clima_info.txt", 'w') as f:
        f.write("Prediction Of Worldwide Energy Resources (POWER) \n")
        info = """

        Fuente: NASA POWER API

        Descripción:
        Este registro de datos climatológicos es una valiosa fuente de información que ofrece un profundo análisis 
        de las condiciones meteorológicas en México. Los datos se obtienen de la API de NASA POWER, que proporciona 
        información detallada sobre diversos parámetros climáticos. Estos datos incluyen, pero no se limitan a, la 
        temperatura, la precipitación, la radiación solar, la velocidad del viento, la humedad relativa y otros 
        indicadores meteorológicos esenciales.

        Cobertura Geográfica:
        Los datos climatológicos se recopilan para los 32 estados y todos los municipios de México (2458), lo que 
        garantiza una cobertura completa de todo el territorio nacional. Esto permite un seguimiento preciso de 
        las condiciones climáticas en áreas urbanas y rurales, así como en zonas remotas.

        Rango de Tiempo:
        Los datos abarcan un período que se extiende desde 2020-01-01 hasta la fecha actual, lo que 
        significa que se pueden acceder a registros históricos y datos en tiempo real. Esto es esencial para el 
        análisis de tendencias a lo largo del tiempo y para la toma de decisiones basadas en datos precisos.

        Parámetros Climáticos:
        El registro incluye una variedad de parámetros climáticos, como la temperatura máxima y mínima diaria, la 
        precipitación acumulada, la radiación solar, la velocidad del viento, la humedad relativa y otros indicadores 
        climáticos relevantes. Estos datos son fundamentales para una amplia gama de aplicaciones, incluyendo la 
        agricultura, la gestión de recursos hídricos, la planificación urbana y la investigación científica.

        Usos Potenciales:
        Este registro de datos climatológicos tiene un amplio rango de aplicaciones, como la optimización de los 
        calendarios de siembra y cosecha en la agricultura, la gestión de la disponibilidad de agua para riego, 
        la planificación de proyectos de energía solar, la evaluación de riesgos climáticos y la toma de decisiones 
        informadas en diversas industrias y sectores.
        
        Este registro se encuentra disponible para consulta y análisis, y puede ser una herramienta valiosa 
        para investigadores, científicos, planificadores y profesionales que requieran datos climatológicos precisos 
        y detallados.

        """ 
        f.write(info + '\n')
        f.write("Descargado el " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
        f.write('\n')
        f.write("Desde: " + 'https://power.larc.nasa.gov/docs/tutorials/api-getting-started/' + "\n")

## Descarga de Datos a nivel Estatal 

In [53]:
coord_estado = pd.read_csv(input_coord_estado) # Lee archivo de entrada coordenadas estado
estado_nom = coord_estado['Estado'].tolist() # Lista de nombres de estado
estado_ids = coord_estado['Estado_ID'].tolist() # Lista de ids de estado
locations = [(lat, lon) for lat, lon in zip(coord_estado['centroid_lat'], coord_estado['centroid_lon'])] # Lista de coordenadas de estado
coord_estado.head(10)

,Estado,Estado_ID,centroid_lat,centroid_lon,Pertenece
0,Zacatecas,MX-ZAC,23.266403,-102.708831,True
1,Yucatán,MX-YUC,20.779755,-88.927237,True
2,Veracruz,MX-VER,19.351889,-96.378018,True
3,Tlaxcala,MX-TLA,19.452613,-98.083483,True
4,Tamaulipas,MX-TAM,24.300779,-98.674508,True
5,Tabasco,MX-TAB,17.932353,-92.618729,True
6,Sonora,MX-SON,29.683748,-110.761555,True
7,Sinaloa,MX-SIN,24.992580,-107.491726,True
8,San Luis Potosí,MX-SLP,22.605843,-100.451600,True
9,Quintana Roo,MX-ROO,19.670401,-88.118478,True


### Definición de Parámetros

- **TS**: Earth Skin Temperature ($°C$) 
- **TS_MAX**: Earth Skin Temperature Maximum ($°C$) 
- **TS_MIN**: Earth Skin Temperature Minimum ($°C$) 
- **T2M**: Temperature at 2 Meters ($°C$) 
- **T2M_MAX**: Temperature at 2 Meters Maximum ($°C$) 
- **T2M_MIN**: Temperature at 2 Meters Minimum ($°C$) 
- **T2MDEW**: Dew/Frost Point at 2 Meters ($°C$) 
- **T2MWET**: Wet Bulb Temperature at 2 Meters ($°C$) 
- **PS**: Surface Pressure ($kPa$) 
- **WS2M**: Wind Speed at 2 Meters ($m/s$) 
- **RH2M**: Relative Humidity at 2 Meters (%) 
- **EVPTRNS**: Evapotranspiration Energy Flux ($kW-hr/m^2/day$) 
- **GWETPROF**: Profile Soil Moisture (%) 
- **CLRSKY_DAYS**: Clear Sky Day ($Days$) 
- **PRECTOTCORR**: Precipitation Corrected ($mm/day$) 
- **SG_DAY_HOURS**: Solar Geometry Day Hours ($Hours$) 
- **MIDDAY_INSOL**: Midday Insolation Incident ($kW-hr/m^2/day$) 

In [46]:
parameter = 'TS,TS_MAX,TS_MIN,T2M,T2M_MAX,T2M_MIN,T2MDEW,T2MWET,PS,WS2M,RH2M,EVPTRNS,GWETPROF,CLRSKY_DAYS,PRECTOTCORR,SG_DAY_HOURS,MIDDAY_INSOL' # Parametros a descargar

start_day = '20200101' # Primer dia del año 2020
end_day =  datetime.datetime.now().strftime('%Y%m%d') # Los Ultimos dias del mes actual vienen con faltantes

base_url = r"https://power.larc.nasa.gov/api/temporal/daily/point?parameters={parameter}&community=RE&longitude={longitude}&latitude={latitude}&start={start_day}&end={end_day}&format=CSV" # URL de consulta

### Descarga de la API

In [22]:
# Descarga de datos por estado
for estado_nom, latitude, longitude in zip(coord_estado['Estado'], coord_estado['centroid_lat'], coord_estado['centroid_lon']): # Itera sobre los estados y sus coordenadas
    api_request_url = base_url.format(longitude=longitude, latitude=latitude, parameter=parameter, start_day=start_day, end_day=end_day) # Forma la URL de consulta

    response = requests.get(url=api_request_url, verify=True, timeout=30.00) # Hace la consulta a la API

    # Si la consulta fue exitosa, guarda el archivo en la carpeta de salida
    if response.status_code == 200:
        content = response.content.decode('utf-8')
        filename = f"Data_{estado_nom}_{start_day}-{end_day}.csv"
        filepath = os.path.join(output_estado_raw, filename)

        with open(filepath, 'wb') as file_object: # Guarda el archivo en la carpeta de salida
            file_object.write(response.content)

print("Datos de estados descargados")

### Dataframe de prueba

In [49]:
# Consulta datos del primer archivo CSV de la carpeta de salida a forma de prueba
archivos_csv = [archivo for archivo in os.listdir(output_estado_raw) if archivo.endswith('.csv')]

if archivos_csv:
    # Lee el primer archivo CSV de la lista
    primer_archivo = archivos_csv[0]
    ruta_completa_primer_csv = os.path.join(output_estado_raw, primer_archivo)
    df_test = pd.read_csv(ruta_completa_primer_csv,skiprows=25)

print(df_test.shape)
df_test

(1403, 20)


,YEAR,MO,DY,TS,TS_MAX,TS_MIN,T2M,T2M_MAX,T2M_MIN,T2MDEW,T2MWET,PS,WS2M,RH2M,EVPTRNS,GWETPROF,CLRSKY_DAYS,PRECTOTCORR,SG_DAY_HOURS,MIDDAY_INSOL
0,2020,1,1,13.37,22.76,6.50,13.30,20.08,7.35,7.12,10.21,79.30,3.91,68.44,0.04,0.43,0.0,6.93,10.0,12.09
1,2020,1,2,10.40,18.46,1.71,10.26,16.07,3.16,5.71,7.99,79.07,4.16,75.94,0.03,0.43,0.0,6.69,10.0,14.65
2,2020,1,3,7.87,21.67,-1.38,7.77,18.33,0.31,-1.83,2.97,79.56,2.52,59.25,0.06,0.44,1.0,0.01,10.0,19.79
3,2020,1,4,6.33,20.80,-3.20,6.06,17.44,-2.65,-8.14,-1.04,80.13,3.05,42.31,0.06,0.43,0.0,0.00,10.0,16.03
4,2020,1,5,7.31,23.21,-3.40,7.55,20.03,-1.74,-8.03,-0.24,80.07,2.41,39.00,0.07,0.43,1.0,0.00,10.0,19.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1398,2023,10,30,20.26,32.43,12.12,19.04,26.80,12.48,10.76,14.90,79.41,1.54,63.56,-999.00,-999.00,-999.0,0.13,-999.0,-999.00
1399,2023,10,31,18.01,29.73,11.17,16.53,23.94,11.35,9.98,13.26,79.58,1.91,68.25,-999.00,-999.00,-999.0,2.75,-999.0,-999.00
1400,2023,11,1,17.10,31.70,8.26,15.33,24.58,8.44,6.79,11.05,79.70,2.19,60.50,-999.00,-999.00,-999.0,0.26,-999.0,-999.00
1401,2023,11,2,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.0,-999.00,-999.0,-999.00


### Transformación de Dataframe

In [ ]:
# Obtén la lista de archivos CSV en la carpeta
archivos_csv = [archivo for archivo in os.listdir(output_estado_raw) if archivo.endswith('.csv')]
n = 0 # Contador de toda vida 

# Procesa y transforma cada archivo CSV
for archivo_csv in archivos_csv:

    # Lee el archivo CSV, salta las primeras 25 filas y agrega una columna con el nombre del estado
    ruta_completa = os.path.join(output_estado_raw, archivo_csv)
    df = pd.read_csv(ruta_completa, skiprows=25)
    estado_nom = archivo_csv.split('_')[1]
    df.insert(0, 'Estado_Nom', estado_nom)

    # Agrega una columna con el ID del estado
    df.insert(1, 'Estado_ID', estado_ids[n])
    n+=1
 
    # Transforma las columnas YEAR, MO y DY en una sola columna de tipo fecha
    df = df.rename(columns={'YEAR': 'year', 'MO': 'month', 'DY': 'day'})
    df.insert(0, 'Fecha', pd.to_datetime(df[['year', 'month', 'day']], format='%Y%m%d'))
    df.drop(columns=['year', 'month', 'day'], inplace=True) 

    # Guarda el archivo CSV procesado en la carpeta de salida
    ruta_completa_salida = os.path.join(output_estado_mod, archivo_csv)
    with open(ruta_completa_salida, 'w') as file_object:
        df.to_csv(file_object, index=False,lineterminator='\n')

### Concatenación de Dataframes

In [25]:
# Obtener la lista de archivos CSV en la carpeta
archivos_csv = [archivo for archivo in os.listdir(output_estado_mod)]

# Lista para almacenar los DataFrames individuales
dataframes = []

# Lee y combina los archivos CSV en un solo DataFrame
for archivo_csv in archivos_csv:
    ruta_completa = os.path.join(output_estado_mod, archivo_csv)
    df = pd.read_csv(ruta_completa)
    dataframes.append(df)

# Combina los DataFrames en uno solo
df_combinado = pd.concat(dataframes, ignore_index=True)

output_filename = 'Data_Climatic_Estados_Concat.csv'  # Nombre del archivo de salida
output_path = os.path.join(output_estado, output_filename)

# Guarda el DataFrame combinado en el archivo CSV utilizando with open
with open(output_path, 'w', newline='') as file_object:
    df_combinado.to_csv(file_object, index=False)

df_combinado

,Fecha,Estado_Nom,Estado_ID,TS,TS_MAX,TS_MIN,T2M,T2M_MAX,T2M_MIN,T2MDEW,T2MWET,PS,WS2M,RH2M,EVPTRNS,GWETPROF,CLRSKY_DAYS,PRECTOTCORR,SG_DAY_HOURS,MIDDAY_INSOL
0,2020-01-01,Aguascalientes,MX-ZAC,13.37,22.76,6.50,13.30,20.08,7.35,7.12,10.21,79.30,3.91,68.44,0.04,0.43,0.0,6.93,10.0,12.09
1,2020-01-02,Aguascalientes,MX-ZAC,10.40,18.46,1.71,10.26,16.07,3.16,5.71,7.99,79.07,4.16,75.94,0.03,0.43,0.0,6.69,10.0,14.65
2,2020-01-03,Aguascalientes,MX-ZAC,7.87,21.67,-1.38,7.77,18.33,0.31,-1.83,2.97,79.56,2.52,59.25,0.06,0.44,1.0,0.01,10.0,19.79
3,2020-01-04,Aguascalientes,MX-ZAC,6.33,20.80,-3.20,6.06,17.44,-2.65,-8.14,-1.04,80.13,3.05,42.31,0.06,0.43,0.0,0.00,10.0,16.03
4,2020-01-05,Aguascalientes,MX-ZAC,7.31,23.21,-3.40,7.55,20.03,-1.74,-8.03,-0.24,80.07,2.41,39.00,0.07,0.43,1.0,0.00,10.0,19.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44891,2023-10-30,Zacatecas,MX-AGU,19.26,30.80,11.84,17.87,25.23,11.58,11.33,14.61,79.88,2.61,69.69,-999.00,-999.00,-999.0,1.16,-999.0,-999.00
44892,2023-10-31,Zacatecas,MX-AGU,13.39,24.35,6.09,11.45,18.00,6.09,6.73,9.09,80.17,3.59,74.50,-999.00,-999.00,-999.0,0.35,-999.0,-999.00
44893,2023-11-01,Zacatecas,MX-AGU,14.76,29.64,5.92,12.65,21.80,5.69,6.27,9.46,80.23,2.41,68.94,-999.00,-999.00,-999.0,0.23,-999.0,-999.00
44894,2023-11-02,Zacatecas,MX-AGU,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.0,-999.00,-999.0,-999.00


## Descarga de Datos a nivel Municipal

In [54]:
coord_municipio = pd.read_csv(input_coord_municipio) # Lee archivo de entrada coordenadas municipio
municipio_nom = coord_municipio['Municipio'].tolist() # Lista de nombres de municipio
locations_mun = [(lat, lon) for lat, lon in zip(coord_municipio['centroid_lat'], coord_municipio['centroid_lon'])]
coord_municipio

,Estado_CVE,Municipio_CVE,Municipio,centroid_lat,centroid_lon,Pertenece
0,1,2,Asientos,22.126737,-102.045605,True
1,1,11,San Francisco de los Romo,22.033110,-102.228397,True
2,1,7,Rincón de Romos,22.267232,-102.340802,True
3,1,8,San José de Gracia,22.146885,-102.525446,True
4,1,5,Jesús María,21.932282,-102.445932,True
...,...,...,...,...,...,...
2453,15,90,Tenango del Valle,19.081422,-99.630837,True
2454,15,106,Toluca,19.295633,-99.666356,True
2455,15,58,Nezahualcóyotl,19.422791,-99.023116,True
2456,9,5,Gustavo A. Madero,19.503795,-99.115742,True


Se utilizan los mismos parámetros y el mismo base_url que en el caso a nivel estatal.

In [32]:
# Mismos parametros que para los estados
# mismo base_url que para los estados

start_day = '20200101' # mismo que para los estados
end_day =  datetime.now().strftime('%Y%m%d')

### Descarga de la API

In [ ]:
# Descarga de datos por municipio

# Itera sobre los municipios y sus coordenadas para hacer la consulta a la API
for estado_cve, municipio_cve, municipio_nom, latitude, longitude in zip(coord_municipio['Estado_CVE'], coord_municipio['Municipio_CVE'], coord_municipio['Municipio'], coord_municipio['centroid_lat'], coord_municipio['centroid_lon']): 
    api_request_url = base_url.format(longitude=longitude, latitude=latitude, parameter=parameter, start_day=start_day, end_day=end_day) # Forma la URL de consulta

    response = requests.get(url=api_request_url, verify=True, timeout=30.00) # Hace la consulta a la API

    # Si la consulta fue exitosa, guarda el archivo en la carpeta de salida
    if response.status_code == 200:
        content = response.content.decode('utf-8')
        filename = f"Data-{estado_cve}-{municipio_cve}_{municipio_nom}_{start_day}-{end_day}.csv"
        filepath = os.path.join(output_municipio_raw, filename)

        with open(filepath, 'wb') as file_object: # Guarda el archivo en la carpeta de salida
            file_object.write(response.content)

print("Datos de estados descargados")

### Dataframe de prueba

In [50]:
# Consulta datos del primer archivo CSV de la carpeta de salida a forma de prueba
archivos_csv = [archivo for archivo in os.listdir(output_municipio_raw) if archivo.endswith('.csv')]

if archivos_csv:
    # Lee el primer archivo CSV de la lista
    primer_archivo = archivos_csv[0]
    ruta_completa_primer_csv = os.path.join(output_municipio_raw, primer_archivo)
    df_test = pd.read_csv(ruta_completa_primer_csv,skiprows=25)

print(df_test.shape)
df_test

(1402, 20)


,YEAR,MO,DY,TS,TS_MAX,TS_MIN,T2M,T2M_MAX,T2M_MIN,T2MDEW,T2MWET,PS,WS2M,RH2M,EVPTRNS,GWETPROF,CLRSKY_DAYS,PRECTOTCORR,SG_DAY_HOURS,MIDDAY_INSOL
0,2020,1,1,12.91,23.01,6.02,12.88,19.87,7.19,7.03,9.95,78.76,5.12,69.81,0.01,0.50,0.0,13.81,10.0,12.09
1,2020,1,2,11.03,19.21,3.86,10.91,16.86,5.05,6.05,8.48,78.48,6.74,74.31,0.01,0.51,0.0,5.85,10.0,14.65
2,2020,1,3,8.24,21.95,0.26,8.14,18.27,1.51,-1.43,3.36,79.02,3.66,59.56,0.02,0.51,1.0,0.02,10.0,19.79
3,2020,1,4,6.03,20.34,-3.12,5.65,16.12,-1.79,-6.48,-0.42,79.68,3.70,49.56,0.01,0.51,0.0,0.00,10.0,16.03
4,2020,1,5,6.23,22.62,-4.48,6.79,18.73,-1.85,-7.67,-0.44,79.60,3.02,42.12,0.02,0.51,1.0,0.00,10.0,19.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397,2023,10,29,21.12,35.68,11.92,19.13,27.32,12.52,12.08,15.60,78.96,1.34,69.88,-999.00,-999.00,-999.0,0.13,-999.0,-999.00
1398,2023,10,30,18.40,30.68,11.73,16.85,24.48,11.74,9.05,12.94,79.00,3.40,67.50,-999.00,-999.00,-999.0,0.20,-999.0,-999.00
1399,2023,10,31,16.31,29.09,8.74,14.51,22.44,8.44,8.58,11.55,79.13,2.99,70.88,-999.00,-999.00,-999.0,1.22,-999.0,-999.00
1400,2023,11,1,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.0,-999.00,-999.0,-999.00


### Transforma el DataFrame

In [ ]:
# Obtén la lista de archivos CSV en la carpeta
archivos_csv = [archivo for archivo in os.listdir(output_municipio_raw) if archivo.endswith('.csv')]

# Procesa y transforma cada archivo CSV
for archivo_csv in archivos_csv:

    # Lee el archivo CSV, salta las primeras 25 filas y agrega una columna con el nombre del estado
    ruta_completa = os.path.join(output_municipio_raw, archivo_csv)
    df = pd.read_csv(ruta_completa, skiprows=25)
    municipio_nom = archivo_csv.split('_')[1]
    df.insert(0, 'Municipio_Nom', municipio_nom)

    estado_cve = archivo_csv.split('_')[0].split('-')[1] # Agrega una columna con el ID del estado
    df.insert(1, 'Estado_CVE', estado_cve)

    municipio_cve = archivo_csv.split('_')[0].split('-')[2] # Agrega una columna con el ID del municipio
    df.insert(2, 'Municipio_CVE', municipio_cve)
 
    # Transforma las columnas YEAR, MO y DY en una sola columna de tipo fecha
    df = df.rename(columns={'YEAR': 'year', 'MO': 'month', 'DY': 'day'})
    df.insert(0, 'Fecha', pd.to_datetime(df[['year', 'month', 'day']], format='%Y%m%d'))
    df.drop(columns=['year', 'month', 'day'], inplace=True) 

    # Guarda el archivo CSV procesado en la carpeta de salida
    ruta_completa_salida = os.path.join(output_municipio_mod, archivo_csv)
    with open(ruta_completa_salida, 'w') as file_object:
        df.to_csv(file_object, index=False,lineterminator='\n')

### Concatena DataFrames

In [37]:
# Obtener la lista de archivos CSV en la carpeta
archivos_csv = [archivo for archivo in os.listdir(output_municipio_mod)]

# Lista para almacenar los DataFrames individuales
dataframes = []

# Lee y combina los archivos CSV en un solo DataFrame
for archivo_csv in archivos_csv:
    ruta_completa = os.path.join(output_municipio_mod, archivo_csv)
    df = pd.read_csv(ruta_completa)
    dataframes.append(df)

# Combina los DataFrames en uno solo
df_combinado = pd.concat(dataframes, ignore_index=True)

output_filename = 'Data_Climatic_Municipios_Concat.csv'  # Nombre del archivo de salida
output_path = os.path.join(output_municipio, output_filename)

# Guarda el DataFrame combinado en el archivo CSV utilizando with open
with open(output_path, 'w', newline='') as file_object:
    df_combinado.to_csv(file_object, index=False)

df_combinado

,Fecha,Municipio_Nom,Estado_CVE,Municipio_CVE,TS,TS_MAX,TS_MIN,T2M,T2M_MAX,T2M_MIN,...,T2MWET,PS,WS2M,RH2M,EVPTRNS,GWETPROF,CLRSKY_DAYS,PRECTOTCORR,SG_DAY_HOURS,MIDDAY_INSOL
0,2020-01-01,El Llano,1,10,12.91,23.01,6.02,12.88,19.87,7.19,...,9.95,78.76,5.12,69.81,0.01,0.50,0.0,13.81,10.0,12.09
1,2020-01-02,El Llano,1,10,11.03,19.21,3.86,10.91,16.86,5.05,...,8.48,78.48,6.74,74.31,0.01,0.51,0.0,5.85,10.0,14.65
2,2020-01-03,El Llano,1,10,8.24,21.95,0.26,8.14,18.27,1.51,...,3.36,79.02,3.66,59.56,0.02,0.51,1.0,0.02,10.0,19.79
3,2020-01-04,El Llano,1,10,6.03,20.34,-3.12,5.65,16.12,-1.79,...,-0.42,79.68,3.70,49.56,0.01,0.51,0.0,0.00,10.0,16.03
4,2020-01-05,El Llano,1,10,6.23,22.62,-4.48,6.79,18.73,-1.85,...,-0.44,79.60,3.02,42.12,0.02,0.51,1.0,0.00,10.0,19.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3446111,2023-10-29,Milpa Alta,9,9,17.36,26.55,11.39,17.51,23.84,12.43,...,15.52,79.16,0.81,79.31,-999.00,-999.00,-999.0,1.07,-999.0,-999.00
3446112,2023-10-30,Milpa Alta,9,9,16.24,24.74,10.30,16.44,22.36,11.41,...,14.11,79.20,1.09,76.50,-999.00,-999.00,-999.0,0.08,-999.0,-999.00
3446113,2023-10-31,Milpa Alta,9,9,15.48,24.34,9.40,15.36,21.87,10.44,...,13.46,79.28,1.26,79.75,-999.00,-999.00,-999.0,0.89,-999.0,-999.00
3446114,2023-11-01,Milpa Alta,9,9,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,...,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.0,-999.00,-999.0,-999.00


# Listo! Por ahora...